# *Импорт библиотек*

In [ ]:
from google.colab import files # Импортируем Модуль files для работы с файлами
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-white')
sns.set_context('notebook', rc={'figure.figsize': (10, 8)}, font_scale=1.5)

import numpy as np # Импортируем библиотеку numpy
import pandas as pd

import time
import random
import math
import re
import os # Импортируем библиотеку os для раоты с фаловой системой
import shutil
import json
import cv2
from google.colab.patches import cv2_imshow
import imutils

from copy import deepcopy
from IPython.display import clear_output

from PIL import ImageFile, ImageOps # импортируем модель Image для работы с изображениями
from PIL import Image as PILImage
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from google.colab import drive  # Подключаем гугл-диск
drive.mount('/content/drive')

In [ ]:
# Контекстный менеджер для измерения времени операций
# Операция обертывается менеджером с помощью оператора with
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        # print('Время обработки: {:.2f} с'.format(time.time() - self.t))
        seconds_total = time.time() - self.t
        minutes = int(seconds_total // 60)
        seconds = (seconds_total % 60) + \
            (seconds_total - math.floor(seconds_total))
        print('Время обработки: {} мин {:.2f} c'.format(minutes, seconds))


with timex():
    print()

# Загрузка датафрейма и составление выборок

In [ ]:
# Загрузка
df = pd.read_csv('/content/drive/MyDrive/df_info_new_ru.csv')

In [ ]:
df.columns

In [ ]:
df = df.sort_values(by='индекс файла')

In [ ]:
print(*df[df['проверка таблиц'] == 1].filename_new[:10], sep='\n')

In [ ]:
def get_img_path(base_path, filename):
    x = int(filename[:-4])
    lower_bound = (x // 100) * 100
    upper_bound = lower_bound + 100
    return base_path + f'{lower_bound}-{upper_bound}/' + filename


# Проверка
base_path = '/content/drive/MyDrive/ClearedReshapedBase/'
x_values = ['0001.png', '0019.png', '0100.png', '0654.png', '1119.png']

for x in x_values:
    y = get_img_path(base_path, x)
    print(y)

In [ ]:
base_path = '/content/drive/MyDrive/ClearedReshapedBase/'
test_filepaths = [get_img_path(base_path, i) for i in df.loc[(
    df['искажение документа'] == 1) & (df['искажение текста'] == 1)]['filename_new']]

In [ ]:
# Составляем выборки файлов для проверки таблиц
base_path = '/content/drive/MyDrive/ClearedReshapedBase/'
target_cols = ['проверка таблиц', 'нестандартный контраст', 'нестандартная верхняя таблица', 'нестандартная нижняя таблица',
               'обрезанный текст', 'обрезанные границы таблиц', 'не белый фон', 'точечный текст', 'нестандартное форматирование текста',
               'плохая печать', 'искажение текста', 'искажение документа', '2 документа', '2 банка в верхней таблице', 'грязь']

sample_dict = {col_name: [get_img_path(base_path, i) for i in list(
    df[df[col_name] == 1].filename_new)] for col_name in target_cols}

# Добавляем выборку без ошибок
rows_with_zeros = df[(df[df.columns[20:]] == 0).all(axis=1)]
filtered_df = df.loc[rows_with_zeros.index]
sample_dict['без ошибок'] = [get_img_path(
    base_path, f'{i:04}.png') for i in list(filtered_df['индекс файла'])[:100]]

In [ ]:
def print_img(image_path, figsize=(12, 8), title=None):
    plt.rcParams['figure.facecolor'] = 'lightgray'
    img = PILImage.open(image_path)
    img = img.convert('RGB')

    # Отображение изображения с помощью Matplotlib
    plt.figure(figsize=figsize)
    plt.imshow(img)
    if title == None:
        plt.title(image_path[-8:])  # Установка заголовка из названия файла
    else:
        plt.title(title)  # Установка заголовка из названия файла
    plt.axis('off')  # Убрать оси координат
    plt.show()

In [ ]:
# def display_images_in_table(image_paths, num_columns=4, image_size=(6, 6), border_color='black', border_width=5, title=False, starting_index=0, show_graph=False):
#     num_images = len(image_paths)
#     num_rows = math.ceil(num_images / num_columns)

#     fig, axes = plt.subplots(num_rows, num_columns, figsize=(num_columns * image_size[0], num_rows * image_size[1]))
#     if title == True:
#         fig.suptitle(title, fontsize=48, y=1.005)
#     else:
#         fig.suptitle(f"{starting_index}-{starting_index+num_images}", fontsize=48, y=1.005)

#     for i, ax in enumerate(axes.flat):
#         if i < num_images:
#             img_path = image_paths[i]
#             img = PILImage.open(img_path)
#             img = img.convert('RGB')

#             # Добавляем обводку вокруг изображения
#             border = Rectangle((0, 0), img.width, img.height, linewidth=border_width, edgecolor=border_color, facecolor='none')
#             ax.add_patch(border)

#             ax.imshow(img)
#             # ax.set_title(f"Изображение {i + 1}", fontsize=24)
#             img_title = img_path[-8:]
#             ax.set_title(f"{img_title} ({starting_index+i})", fontsize=24)
#             ax.axis('off')
#         else:
#             ax.axis('off')

#     plt.tight_layout()
#     if show_graph == True:
#         plt.show()
#     else:
#         plt.savefig(f"/content/drive/MyDrive/test_img/{starting_index}-{starting_index+num_images}.png")
#         # plt.savefig(f"/content/drive/MyDrive/imgs/{starting_index}-{starting_index+50}.png")

#     plt.clf()

# for i in range(len(test_filepaths) // 100 + 1):
#     print(i)
#     with timex():
#         display_images_in_table(test_filepaths[100*i:100*(i+1)], num_columns=4, image_size=(12, 12), border_color='blue', border_width=5, title=False, starting_index=100*i, show_graph=False)

In [ ]:
# test_filepaths = sample_dict['нестандартное форматирование текста']
# for i in range(len(test_filepaths) // 100 + 1):
#     print(i)
#     with timex():
#         display_images_in_table(test_filepaths[100*i:100*(i+1)], num_columns=4, image_size=(6, 6), border_color='blue', border_width=5, title=False, starting_index=100*i, show_graph=False)
#     break

In [ ]:
for key, val in sample_dict.items():
    print(len(val),  '-', key)

In [ ]:
filenums_dict = {'искажение по вертикали': [1654, 1666, 1931, 2089, 2228, 2384, 2408,
                                            2481, 2602, 2759, 2825, 2888, 2938, 2972, 2981, 3038, 3069, 3093, 3167, 3198, 3230, 3243,
                                            3307, 3337, 3339, 3360, 3441, 3445, 3448, 3456, 3466, 3475, 3485, 3493, 3494, 3511, 3530,
                                            3531, 3542, 3709, 3710, 3729, 3780, 4210, 4212, 4240, 4241, 4242, 4243, 4261, 4264, 4280,
                                            4281, 4293, 4304, 4313, 4328, 4325, 4322, 4331, 4343, 4347, 4353, 4354, 4359, 4363, 4373,],
                 'плохая печать': [88, 115, 148, 205, 259, 297, 316, 344, 480,
                                   487, 511, 591, 741, 751, 811, 928, 990, 999, 1080, 1106, 1107, 1118, 1225, 1248,
                                   1306, 1395, 1585, 1753, 1910, 1915, 1929, 1974, 1991, 2022, 2044, 2050, 2167, 2218, 2223,
                                   2252, 2295, 2308, 2314, 2377, 2379, 2404, 2422, 2427, 2445, 2453, 2478, 2519, 2738, 2746,
                                   2794, 2861, 2889, 3176, 3187, 3226, 3229, 3258, 3289, 3297, 3314, 3357, 3433, 3567, 3635,
                                   3659, 3684, 3703, 3713, 3748, 3776, 3829, 3853, 3854, 4027, 4041, 4051, 4121, 4159, 4170,
                                   4249, 4257, 4327, 4303, 4286, 4284, 4336, 4368, 4488, 4487, 4614, 4655, 4769, 4768, 4689,
                                   4910, 4912, 4979, 4990, 5094, 5066, 5257, 5212, 5261, 5391, 5320, 5500, 5539, 5541, 5544,
                                   5555, 5590, 5581, 5557, 5640, 5654, 5656, 5667, 5663, 5683, 5688, 5736, 5759, 5827, 5927,
                                   5930,
                                   ],
                 'точечный текст': [int(i[-8:-4]) for i in sample_dict['точечный текст']],
                 'не белый фон': [int(i[-8:-4]) for i in sample_dict['не белый фон']],
                 'грязь': [4, 59, 61, 65, 67, 78, 79, 83, 93, 98, 105, 110, 120, 152, 171, 282,
                           329, 350, 413, 695, 1477, 1546, 1608, 1691, 1795, 1805, 1844, 1869, 1894, 1907, 2085, 2122, 2105,
                           2156, 2161, 2183, 2162, 2213, 2322, 2341, 2405, 2406, 2412, 2429, 2443, 2498, 2514, 2541, 2586,
                           2595, 2600, 2618, 2623, 2638, 2640, 2645, 2651, 2656, 2657, 2695, 2715, 2718, 2731, 2733, 2737,
                           2745, 2781, 2782, 2800, 2823, 2906, 3074, 3103, 3115, 3173, 3286, 3318, 3319, 3320, 3330, 3331,
                           3336, 3338, 3350, 3356, 3431, 3451, 3548, 3602, 3617, 3698, 3730, 3775, 3803, 3879, 3888, 3886,
                           3906, 3913, 3915, 3937, 3943, 3954, 3947, 4006, 4011, 4041, 4029, 4028, 4060, 4057, 4055, 4064,
                           4068, 4085, 4093, 4096, 4104, 4118, 4115, 4121, 4130, 4139, 4134, 4131, 4145, 4166, 4180, 4188,
                           4189, 4192, 4200, 4195, 4219, 4220, 4222, 4233, 4247, 4268, 4258, 4285, 4367, 4398, 4402, 4427,
                           4442, 4449, 4476, 4475, 4461, 4477, 4508, 4490, 4530, 4534, 4547, 4608, 4595, 4590, 4650, 4663,
                           4730, 4747, 4748, 4749, 4776, 4782, 4795, 4807, 4839, 4912, 4910, 4937, 4989, 4967, 5048, 5031,
                           5099, 5103, 5173, 5171, 5205, 5207, 5229, 5255, 5264, 5277, 5319, 5307, 5340, 5352, 5368, 5367,
                           5378, 5384, 5398, 5402, 5403, 5419, 5432, 5441, 5442, 5449, 5453, 5459, 5460, 5506, 5496, 5518,
                           5519, 5521, 5532, 5541, 5543, 5544, 5548, 5546, 5555, 5556, 5560, 5569, 5587, 5584, 5579, 5597,
                           5608, 5612, 5633, 5623, 5635, 5645, 5657, 5656, 5648, 5659, 5661, 5662, 5668, 5671, 5672, 5675,
                           5677, 5690, 5688, 5686, 5681, 5696, 5699, 5702, 5712, 5709, 5707, 5730, 5734, 5732, 5739, 5737,
                           5736, 5735, 5755, 5768, 5777, 5799, 5807, 5810, 5818, 5823, 5827, 5831, 5840, 5839, 5855, 5878,
                           5970, 5965, 5974,
                           ],
                 "нестандартное форматирование текста": [87, 94, 148, 164, 178, 200, 201,
                                                         205, 214, 231, 237, 245, 251, 252, 259, 272, 330, 328, 310, 309, 390, 392, 603, 461, 456, 621, 679,
                                                         716, 743, 853, 953, 969, 1016, 1053, 1184, 1205, 1223, 1232, 1268, 1344, 1334, 1423, 1514, 1531,
                                                         1519, 1587, 1711, 1697, 1674, 1745, 1897, 1987, 1986, 2191, 2230, 2231, 2258, 2340, 2412, 2405,
                                                         2429, 2542, 2683, 2583, 2750, 2910, 2916, 3012, 3026, 3083, 3066, 3145, 3156, 3165, 3232, 3197,
                                                         3182, 3292, 3521, 3571, 3568, 3597, 3793, 3922, 3876, 3929, 3935, 4030, 4022, 4021, 4015, 4320,
                                                         4555, 4730, 4649, 5227, 5466, 5703, 5813, 5821,
                                                         ],
                 'нестандартный контраст': [int(i[-8:-4]) for i in sample_dict['нестандартный контраст'][:80]],
                 }

In [ ]:
filenums_dict = {'искажение по вертикали': [1654, 1666, 1931, 2089, 2228, 2384, 2408, 2481, 2602, 2759, 2825, 2888, 2938, 2972, 2981, 3038, 3069, 3093, 3167, 3198, 3230, 3243, 3307, 3337, 3339, 3360, 3441, 3445, 3448, 3456, 3466, 3475, 3485, 3493, 3494, 3511, 3530, 3531, 3542, 3709, 3710, 3729, 3780, 4210, 4212, 4240, 4241, 4242, 4243, 4261, 4264, 4280, 4281, 4293, 4304, 4313, 4328, 4325, 4322, 4331, 4343, 4347, 4353, 4354, 4359, 4363, 4373],
                 'плохая печать': [88, 115, 148, 205, 259, 297, 316, 344, 480, 487, 511, 591, 741, 751, 811, 928, 990, 999, 1080, 1106, 1107, 1118, 1225, 1248, 1306, 1395, 1585, 1753, 1910, 1915, 1929, 1974, 1991, 2022, 2044, 2050, 2167, 2218, 2223, 2252, 2295, 2308, 2314, 2377, 2379, 2404, 2422, 2427, 2445, 2453, 2478, 2519, 2738, 2746, 2794, 2861, 2889, 3176, 3187, 3226, 3229, 3258, 3289, 3297, 3314, 3357, 3433, 3567, 3635, 3659, 3684, 3703, 3713, 3748, 3776, 3829, 3853, 3854, 4027, 4041, 4051, 4121, 4159, 4170, 4249, 4257, 4327, 4303, 4286, 4284, 4336, 4368, 4488, 4487, 4614, 4655, 4769, 4768, 4689, 4910, 4912, 4979, 4990, 5094, 5066, 5257, 5212, 5261, 5391, 5320, 5500, 5539, 5541, 5544, 5555, 5590, 5581, 5557, 5640, 5654, 5656, 5667, 5663, 5683, 5688, 5736, 5759, 5827, 5927, 5930],
                 'точечный текст': [100, 122, 134, 153, 155, 162, 167, 176, 186, 188, 215, 226, 271, 326, 331, 344, 377, 404, 424, 426, 427, 459, 462, 468, 469, 485, 503, 516, 537, 565, 569, 584, 586, 591, 600, 656, 666, 785, 789, 805, 852, 894, 949, 960, 974, 990, 1025, 1027, 1042, 1093, 1106, 1110, 1118, 1151, 1234, 1238, 1248, 1280, 1350, 1359, 1379, 1408, 1548, 1550, 1568, 1585, 1608, 1655, 1661, 1705, 1782, 1826, 1831, 1881, 1922, 1973, 2026, 2054, 2074, 2083, 2114, 2131, 2141, 2164, 2182, 2207, 2223, 2227, 2246, 2302, 2306, 2323, 2352, 2357, 2358, 2379, 2395, 2436, 2443, 2451, 2477, 2481, 2496, 2529, 2530, 2556, 2563, 2580, 2616, 2700, 2701, 2711, 2742, 2780, 2781, 2842, 2888, 2978, 3095, 3097, 3191, 3252, 3276, 3290, 3303, 3342, 3346, 3391, 3404, 3416, 3417, 3418, 3507, 3514, 3543, 3544, 3567, 3578, 3582, 3641, 3654, 3668, 3669, 3706, 3713, 3715, 3721, 3752, 3758, 3773, 3783, 3801, 3825, 3881, 3911, 3914, 3932, 3943, 3956, 4112, 4156, 4158, 4205, 4211, 4215, 4286, 4303, 4337, 4366, 4368, 4376, 4413, 4414, 4431, 4439, 4471, 4494, 4552, 4626, 4662, 4742, 4907, 5098, 5099, 5102, 5301, 5302, 5303, 5404, 5411, 5442, 5529, 5531],
                 'не белый фон': [3391, 3886, 5521, 5587, 5597, 5627, 5657, 5659, 5662, 5671, 5672, 5687, 5688, 5690, 5695, 5699, 5707, 5709, 5712, 5716, 5732, 5735, 5737, 5744, 5746, 5747, 5748, 5749, 5750, 5752, 5753, 5754, 5757, 5762, 5765, 5767, 5769, 5774, 5775, 5777, 5783, 5784, 5789, 5793, 5794, 5795, 5796, 5801, 5802, 5804, 5807, 5811, 5814, 5815, 5822, 5823, 5830, 5846, 5888, 5945, 5947, 5954, 5957, 5958, 5959, 5962, 5963, 5964, 5965, 5966, 5972, 5973, 5974, 5975, 5977, 5978, 5979],
                 'грязь': [4, 59, 61, 65, 67, 78, 79, 83, 93, 98, 105, 110, 120, 152, 171, 282, 329, 350, 413, 695, 1477, 1546, 1608, 1691, 1795, 1805, 1844, 1869, 1894, 1907, 2085, 2122, 2105, 2156, 2161, 2183, 2162, 2213, 2322, 2341, 2405, 2406, 2412, 2429, 2443, 2498, 2514, 2541, 2586, 2595, 2600, 2618, 2623, 2638, 2640, 2645, 2651, 2656, 2657, 2695, 2715, 2718, 2731, 2733, 2737, 2745, 2781, 2782, 2800, 2823, 2906, 3074, 3103, 3115, 3173, 3286, 3318, 3319, 3320, 3330, 3331, 3336, 3338, 3350, 3356, 3431, 3451, 3548, 3602, 3617, 3698, 3730, 3775, 3803, 3879, 3888, 3886, 3906, 3913, 3915, 3937, 3943, 3954, 3947, 4006, 4011, 4041, 4029, 4028, 4060, 4057, 4055, 4064, 4068, 4085, 4093, 4096, 4104, 4118, 4115, 4121, 4130, 4139, 4134, 4131, 4145, 4166, 4180, 4188, 4189, 4192, 4200, 4195, 4219, 4220, 4222, 4233, 4247, 4268, 4258, 4285, 4367, 4398, 4402, 4427, 4442, 4449, 4476, 4475, 4461, 4477, 4508, 4490, 4530, 4534, 4547, 4608, 4595, 4590, 4650, 4663, 4730, 4747, 4748, 4749, 4776, 4782, 4795, 4807, 4839, 4912, 4910, 4937, 4989, 4967, 5048, 5031, 5099, 5103, 5173, 5171, 5205, 5207, 5229, 5255, 5264, 5277, 5319, 5307, 5340, 5352, 5368, 5367, 5378, 5384, 5398, 5402, 5403, 5419, 5432, 5441, 5442, 5449, 5453, 5459, 5460, 5506, 5496, 5518, 5519, 5521, 5532, 5541, 5543, 5544, 5548, 5546, 5555, 5556, 5560, 5569, 5587, 5584, 5579, 5597, 5608, 5612, 5633, 5623, 5635, 5645, 5657, 5656, 5648, 5659, 5661, 5662, 5668, 5671, 5672, 5675, 5677, 5690, 5688, 5686, 5681, 5696, 5699, 5702, 5712, 5709, 5707, 5730, 5734, 5732, 5739, 5737, 5736, 5735, 5755, 5768, 5777, 5799, 5807, 5810, 5818, 5823, 5827, 5831, 5840, 5839, 5855, 5878, 5970, 5965, 5974],
                 'нестандартное форматирование текста': [87, 94, 148, 164, 178, 200, 201, 205, 214, 231, 237, 245, 251, 252, 259, 272, 330, 328, 310, 309, 390, 392, 603, 461, 456, 621, 679, 716, 743, 853, 953, 969, 1016, 1053, 1184, 1205, 1223, 1232, 1268, 1344, 1334, 1423, 1514, 1531, 1519, 1587, 1711, 1697, 1674, 1745, 1897, 1987, 1986, 2191, 2230, 2231, 2258, 2340, 2412, 2405, 2429, 2542, 2683, 2583, 2750, 2910, 2916, 3012, 3026, 3083, 3066, 3145, 3156, 3165, 3232, 3197, 3182, 3292, 3521, 3571, 3568, 3597, 3793, 3922, 3876, 3929, 3935, 4030, 4022, 4021, 4015, 4320, 4555, 4730, 4649, 5227, 5466, 5703, 5813, 5821],
                 'нестандартный контраст': [86, 94, 99, 115, 134, 148, 153, 155, 162, 164, 167, 173, 178, 186, 188, 201, 205, 237, 245, 252, 259, 267, 271, 272, 276, 277, 297, 298, 316, 326, 331, 344, 377, 392, 395, 404, 424, 426, 427, 446, 459, 468, 469, 480, 487, 498, 503, 537, 565, 569, 576, 579, 584, 585, 586, 587, 591, 600, 656, 666, 689, 703, 709, 741, 751, 753, 782, 785, 789, 805, 811, 812, 840, 852, 858, 877, 887, 894, 919, 949],
                 'без ошибок': [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 27, 28, 29, 30, 31, 36, 38, 43, 44, 45, 46, 47, 48, 51, 53, 54, 55, 57, 62, 63, 64, 66, 69, 72, 73, 76, 81, 84, 85, 89, 92, 95, 96, 97, 106, 111, 116, 117, 119, 124, 125, 126, 127, 128, 129, 130, 131, 132, 135, 136, 137, 139, 140, 141, 143, 144, 145, 146, 147, 149, 154, 156, 157, 158, 160, 161, 163, 165, 166, 168, 169, 170, 172, 174, 177, 179, 183, 185,]}

In [ ]:
for i, j in filenums_dict.items():
    print(f"'{i}':{j},")

In [ ]:
def get_img_path(base_path, filename):
    x = int(filename[:-4])
    lower_bound = (x // 100) * 100
    upper_bound = lower_bound + 100
    return base_path + f'{lower_bound}-{upper_bound}/' + filename


# Путь к папке с файлами на google диске
base_path = '/content/drive/MyDrive/ClearedReshapedBase/'

filepaths_dict = {i: [get_img_path(base_path, '{:04}.png'.format(
    k)) for k in j] for i, j in filenums_dict.items()}

# Предобработка тест

In [ ]:
from matplotlib.patches import Rectangle


def display_images_in_table_2(img_names_list, image_list, num_columns=4, image_size=(6, 6), border_color='black', border_width=5, title=False):
    num_images = len(image_list)
    num_rows = 1

    fig, axes = plt.subplots(num_rows, num_columns, figsize=(
        num_columns * image_size[0], num_rows * image_size[1]))
    if title == True:
        fig.suptitle(title, fontsize=48, y=1.005)
    else:
        fig.suptitle(f"Изображение", fontsize=48, y=1.005)

    for i, ax in enumerate(axes.flat):
        if i < num_images:
            img = image_list[0]

            # Добавляем обводку вокруг изображения
            border = Rectangle((0, 0), img.width, img.height,
                               linewidth=border_width, edgecolor=border_color, facecolor='none')
            ax.add_patch(border)

            ax.imshow(img)
            ax.set_title(img_names_list[i], fontsize=24)
            ax.axis('off')
        else:
            ax.axis('off')

    plt.tight_layout()
    plt.show()
    plt.clf()

In [ ]:
def print_img(img, image_path, figsize=(12, 8), title=None):
    # Отображение изображения с помощью Matplotlib
    plt.rcParams['figure.facecolor'] = 'lightgray'
    plt.figure(figsize=figsize)
    plt.imshow(img)
    if title == None:
        plt.title(image_path[-8:])  # Установка заголовка из названия файла
    else:
        plt.title(title)  # Установка заголовка из названия файла
    plt.axis('off')  # Убрать оси координат
    plt.show()


def to_numpy(im):
    im.load()
    # unpack data
    e = PILImage._getencoder(im.mode, 'raw', im.mode)
    e.setimage(im.im)

    # NumPy buffer for the result
    shape, typestr = PILImage._conv_type_shape(im)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))

    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)
        mem[offset:offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data

In [ ]:
display_images_in_table_2(image_path[-8:])

In [ ]:
filepath = test_filepaths[0]
img = cv2.imread(filepath)
cv2_imshow(img)

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cv2_imshow(img)

In [ ]:
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]


def remove_noise(image):
    return cv2.fastNlMeansDenoising(image, h=3, templateWindowSize=7, searchWindowSize=21)


def normalize_img(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    return cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)


def erode_img(img):
    # img = cv2.imread('j.png',0)
    kernel = np.ones((5, 5), np.uint8)
    return cv2.erode(img, kernel, iterations=1)


def blur_img(img):
    return cv2.GaussianBlur(img, (3, 3), 0)


def dilate_img(img):
    return cv2.dilate(img, (10, 10), iterations=3)


def custom_edit_1(img):
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    img = cv2.dilate(img, (10, 10), iterations=3)

    return img


preform_dict = {'thresholding': thresholding,
                'remove_noise': remove_noise,
                'normalize_img': normalize_img,
                'erode_img': erode_img,
                'blur_img': blur_img,
                'dilate_img': dilate_img,

                'blur+threshold+dilate': custom_edit_1,
                }

In [ ]:
for i, j in preform_dict.items():
    print_img(cv2.cvtColor(j(image), cv2.COLOR_GRAY2RGB),
              filepath, figsize=(20, 16), title=i)

In [ ]:
filepath = test_filepaths[0]
image = cv2.imread(filepath, cv2.COLOR_RGB2GRAY)  # cv2.COLOR_BGR2GRAY

print_img(image, filepath, figsize=(12, 8), title=None)

image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
dist = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
dist = cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
dist = (dist * 255).astype("uint8")
dist = cv2.threshold(dist, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

print_img(dist, filepath, figsize=(12, 8), title=None)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
opening = cv2.morphologyEx(dist, cv2.MORPH_OPEN, kernel)

print_img(opening, filepath, figsize=(12, 8), title=None)

cnts = cv2.findContours(opening.copy(), cv2.RETR_EXTERNAL,
                        cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
chars = []
# loop over the contours
for c in cnts:
    # compute the bounding box of the contour
    (x, y, w, h) = cv2.boundingRect(c)
    # check if contour is at least 35px wide and 100px tall, and if
    # so, consider the contour a digit
    if w >= 35 and h >= 100:
        chars.append(c)

chars = np.vstack([chars[i] for i in range(0, len(chars))])
hull = cv2.convexHull(chars)

mask = np.zeros(image.shape[:2], dtype="uint8")
cv2.drawContours(mask, [hull], -1, 255, -1)
mask = cv2.dilate(mask, None, iterations=2)

print_img(mask, filepath, figsize=(12, 8), title=None)

final = cv2.bitwise_and(opening, opening, mask=mask)

print_img(final, filepath, figsize=(12, 8), title=None)

In [ ]:
filepath = test_filepaths[0]
img = PILImage.open(filepath)
img = img.convert('RGB')

img_cv2 = to_numpy(img)
img_cv2 = cv2.cvtColor(img_cv2, cv2.COLOR_RGB2GRAY)
blur = cv2.GaussianBlur(img_cv2, (3, 3), 0)
_, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
dilation = cv2.dilate(thresh, (10, 10), iterations=3)
contours, _ = cv2.findContours(
    dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print_img(img, filepath, figsize=(12, 8), title=None)
blur = cv2.cvtColor(blur, cv2.COLOR_GRAY2RGB)
print_img(blur, filepath, figsize=(12, 8), title='blur')
thresh = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
print_img(thresh, filepath, figsize=(12, 8), title='thresh')
dilation = cv2.cvtColor(dilation, cv2.COLOR_GRAY2RGB)
print_img(dilation, filepath, figsize=(12, 8), title='dilition')
# print_img(contours, filepath, figsize=(12,8), title=None)

# **EasyOCR тест предобработки**

### ***Установка библиотек, и составление функций***

In [ ]:
%%capture
!pip install easyocr
import easyocr

In [ ]:
from IPython.display import display_html  # , display
# display_html

In [ ]:
# Создаём EasyOCR
reader = easyocr.Reader(['ru', 'en'])

In [ ]:
filepaths_dict.keys()

dict_keys(['искажение по вертикали', 'плохая печать', 'точечный текст', 'не белый фон', 'грязь', 'нестандартное форматирование текста', 'нестандартный контраст'])

In [ ]:
for key, val in filepaths_dict.items():
    print(f'{len(val):4}',  '-', key)

In [ ]:
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]


def remove_noise(image):
    return cv2.fastNlMeansDenoising(image, h=3, templateWindowSize=7, searchWindowSize=21)


def normalize_img(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    return cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)


def erode_img(img):
    # img = cv2.imread('j.png',0)
    kernel = np.ones((5, 5), np.uint8)
    return cv2.erode(img, kernel, iterations=1)


def blur_img(img):
    return cv2.GaussianBlur(img, (3, 3), 0)


def dilate_img(img):
    return cv2.dilate(img, (10, 10), iterations=3)


def custom_edit_1(img):
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    img = cv2.dilate(img, (10, 10), iterations=3)

    return img


preform_dict = {'thresholding': thresholding,
                'remove_noise': remove_noise,
                'normalize_img': normalize_img,
                'erode_img': erode_img,
                'blur_img': blur_img,
                'dilate_img': dilate_img,

                'blur+threshold+dilate': custom_edit_1,
                }

In [ ]:
def print_img(img, image_path, figsize=(12, 8), title=None):
    # Отображение изображения с помощью Matplotlib
    plt.figure(figsize=figsize)
    plt.imshow(img)
    if title == None:
        plt.title(image_path[-8:])  # Установка заголовка из названия файла
    else:
        plt.title(title)  # Установка заголовка из названия файла
    plt.axis('off')  # Убрать оси координат
    plt.show()


def print_result_string(result_list, name='Default'):
    print(f'{name:>25} ({len(result_list)}):', end=' ')
    print(*result_list[:50], sep=' ')

### ***Тест изображений из каждой категории***

#### **без ошибок**

In [ ]:
category = 'без ошибок'
test_num = 10

cat_len = len(filepaths_dict[category][:test_num])
for i, filepath in enumerate(filepaths_dict[category][:test_num]):
    print('Файл:', filepath[-8:], f'({i+1}/{cat_len}) ({i})')
    with timex():
        img = cv2.imread(filepath)
        print_img(img, filepath, figsize=(12, 8))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Находим текст без предобработки
        names_list = ['Default']
        results_list = []
        results_list.append(reader.readtext(img, detail=0))

        # Находим текст с предобработкой
        for preform_name, preform in preform_dict.items():
            names_list.append(preform_name)
            results_list.append(reader.readtext(preform(img), detail=0))

        df = pd.DataFrame(results_list, index=names_list)
        df = df.T
        display_html(df.head(25).to_html(), raw=True)

        for name, res in zip(names_list, results_list):
            print_result_string(res, name)

    print()

#### **искажение по вертикали**

In [ ]:
category = 'искажение по вертикали'

cat_len = len(filepaths_dict[category])
for i, filepath in enumerate(filepaths_dict[category]):
    print('Файл:', filepath[-8:], f'({i+1}/{cat_len})')
    with timex():
        img = cv2.imread(filepath)
        print_img(img, filepath, figsize=(12, 8))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Находим текст без предобработки
        names_list = ['Default']
        results_list = []
        results_list.append(reader.readtext(img, detail=0))

        # Находим текст с предобработкой
        for preform_name, preform in preform_dict.items():
            names_list.append(preform_name)
            results_list.append(reader.readtext(preform(img), detail=0))

        df = pd.DataFrame(results_list, index=names_list)
        df = df.T
        display_html(df.head(25).to_html(), raw=True)

        for name, res in zip(names_list, results_list):
            print_result_string(res, name)

    print()

#### **не белый фон**

In [ ]:
category = 'не белый фон'
test_num = 30

cat_len = len(filepaths_dict[category][:test_num])
for i, filepath in enumerate(filepaths_dict[category][:test_num]):
    print('Файл:', filepath[-8:], f'({i+1}/{cat_len}) ({i})')
    with timex():
        img = cv2.imread(filepath)
        print_img(img, filepath, figsize=(12, 8))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Находим текст без предобработки
        names_list = ['Default']
        results_list = []
        results_list.append(reader.readtext(img, detail=0))

        # Находим текст с предобработкой
        for preform_name, preform in preform_dict.items():
            names_list.append(preform_name)
            results_list.append(reader.readtext(preform(img), detail=0))

        df = pd.DataFrame(results_list, index=names_list)
        df = df.T
        display_html(df.head(25).to_html(), raw=True)

        for name, res in zip(names_list, results_list):
            print_result_string(res, name)

    print()

#### **плохая печать**

In [ ]:
category = 'плохая печать'
test_num = 30

cat_len = len(filepaths_dict[category][:test_num])
for i, filepath in enumerate(filepaths_dict[category][:test_num]):
    print('Файл:', filepath[-8:], f'({i+1}/{cat_len}) ({i})')
    with timex():
        img = cv2.imread(filepath)
        print_img(img, filepath, figsize=(12, 8))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Находим текст без предобработки
        names_list = ['Default']
        results_list = []
        results_list.append(reader.readtext(img, detail=0))

        # Находим текст с предобработкой
        for preform_name, preform in preform_dict.items():
            names_list.append(preform_name)
            results_list.append(reader.readtext(preform(img), detail=0))

        df = pd.DataFrame(results_list, index=names_list)
        df = df.T
        display_html(df.head(25).to_html(), raw=True)

        for name, res in zip(names_list, results_list):
            print_result_string(res, name)

    print()

#### **точечный текст**

In [ ]:
category = 'точечный текст'
test_num = 30

cat_len = len(filepaths_dict[category][:test_num])
for i, filepath in enumerate(filepaths_dict[category][:test_num]):
    print('Файл:', filepath[-8:], f'({i+1}/{cat_len}) ({i})')
    with timex():
        img = cv2.imread(filepath)
        print_img(img, filepath, figsize=(12, 8))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Находим текст без предобработки
        names_list = ['Default']
        results_list = []
        results_list.append(reader.readtext(img, detail=0))

        # Находим текст с предобработкой
        for preform_name, preform in preform_dict.items():
            names_list.append(preform_name)
            results_list.append(reader.readtext(preform(img), detail=0))

        df = pd.DataFrame(results_list, index=names_list)
        df = df.T
        display_html(df.head(25).to_html(), raw=True)

        for name, res in zip(names_list, results_list):
            print_result_string(res, name)

    print()

#### **грязь**

In [ ]:
category = 'грязь'
test_num = 30

cat_len = len(filepaths_dict[category][:test_num])
for i, filepath in enumerate(filepaths_dict[category][:test_num]):
    print('Файл:', filepath[-8:], f'({i+1}/{cat_len}) ({i})')
    with timex():
        img = cv2.imread(filepath)
        print_img(img, filepath, figsize=(12, 8))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Находим текст без предобработки
        names_list = ['Default']
        results_list = []
        results_list.append(reader.readtext(img, detail=0))

        # Находим текст с предобработкой
        for preform_name, preform in preform_dict.items():
            names_list.append(preform_name)
            results_list.append(reader.readtext(preform(img), detail=0))

        df = pd.DataFrame(results_list, index=names_list)
        df = df.T
        display_html(df.head(25).to_html(), raw=True)

        for name, res in zip(names_list, results_list):
            print_result_string(res, name)

    print()

#### **нестандартное форматирование текста**

In [ ]:
category = 'нестандартное форматирование текста'
test_num = 10

cat_len = len(filepaths_dict[category][:test_num])
for i, filepath in enumerate(filepaths_dict[category][:test_num]):
    print('Файл:', filepath[-8:], f'({i+1}/{cat_len}) ({i})')
    with timex():
        img = cv2.imread(filepath)
        print_img(img, filepath, figsize=(12, 8))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Находим текст без предобработки
        names_list = ['Default']
        results_list = []
        results_list.append(reader.readtext(img, detail=0))

        # Находим текст с предобработкой
        for preform_name, preform in preform_dict.items():
            names_list.append(preform_name)
            results_list.append(reader.readtext(preform(img), detail=0))

        df = pd.DataFrame(results_list, index=names_list)
        df = df.T
        display_html(df.head(25).to_html(), raw=True)

        for name, res in zip(names_list, results_list):
            print_result_string(res, name)

    print()

#### **нестандартный контраст**

In [ ]:
category = 'нестандартный контраст'
test_num = 10

cat_len = len(filepaths_dict[category][:test_num])
for i, filepath in enumerate(filepaths_dict[category][:test_num]):
    print('Файл:', filepath[-8:], f'({i+1}/{cat_len}) ({i})')
    with timex():
        img = cv2.imread(filepath)
        print_img(img, filepath, figsize=(12, 8))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Находим текст без предобработки
        names_list = ['Default']
        results_list = []
        results_list.append(reader.readtext(img, detail=0))

        # Находим текст с предобработкой
        for preform_name, preform in preform_dict.items():
            names_list.append(preform_name)
            results_list.append(reader.readtext(preform(img), detail=0))

        df = pd.DataFrame(results_list, index=names_list)
        df = df.T
        display_html(df.head(25).to_html(), raw=True)

        for name, res in zip(names_list, results_list):
            print_result_string(res, name)

    print()

# **Выборки**

In [ ]:
filenums_dict = {'искажение по вертикали': [1654, 1666, 1931, 2089, 2228, 2384, 2408, 2481, 2602, 2759, 2825, 2888, 2938, 2972, 2981, 3038, 3069, 3093, 3167, 3198, 3230, 3243, 3307, 3337, 3339, 3360, 3441, 3445, 3448, 3456, 3466, 3475, 3485, 3493, 3494, 3511, 3530, 3531, 3542, 3709, 3710, 3729, 3780, 4210, 4212, 4240, 4241, 4242, 4243, 4261, 4264, 4280, 4281, 4293, 4304, 4313, 4328, 4325, 4322, 4331, 4343, 4347, 4353, 4354, 4359, 4363, 4373],
                 'плохая печать': [88, 115, 148, 205, 259, 297, 316, 344, 480, 487, 511, 591, 741, 751, 811, 928, 990, 999, 1080, 1106, 1107, 1118, 1225, 1248, 1306, 1395, 1585, 1753, 1910, 1915, 1929, 1974, 1991, 2022, 2044, 2050, 2167, 2218, 2223, 2252, 2295, 2308, 2314, 2377, 2379, 2404, 2422, 2427, 2445, 2453, 2478, 2519, 2738, 2746, 2794, 2861, 2889, 3176, 3187, 3226, 3229, 3258, 3289, 3297, 3314, 3357, 3433, 3567, 3635, 3659, 3684, 3703, 3713, 3748, 3776, 3829, 3853, 3854, 4027, 4041, 4051, 4121, 4159, 4170, 4249, 4257, 4327, 4303, 4286, 4284, 4336, 4368, 4488, 4487, 4614, 4655, 4769, 4768, 4689, 4910, 4912, 4979, 4990, 5094, 5066, 5257, 5212, 5261, 5391, 5320, 5500, 5539, 5541, 5544, 5555, 5590, 5581, 5557, 5640, 5654, 5656, 5667, 5663, 5683, 5688, 5736, 5759, 5827, 5927, 5930],
                 'точечный текст': [100, 122, 134, 153, 155, 162, 167, 176, 186, 188, 215, 226, 271, 326, 331, 344, 377, 404, 424, 426, 427, 459, 462, 468, 469, 485, 503, 516, 537, 565, 569, 584, 586, 591, 600, 656, 666, 785, 789, 805, 852, 894, 949, 960, 974, 990, 1025, 1027, 1042, 1093, 1106, 1110, 1118, 1151, 1234, 1238, 1248, 1280, 1350, 1359, 1379, 1408, 1548, 1550, 1568, 1585, 1608, 1655, 1661, 1705, 1782, 1826, 1831, 1881, 1922, 1973, 2026, 2054, 2074, 2083, 2114, 2131, 2141, 2164, 2182, 2207, 2223, 2227, 2246, 2302, 2306, 2323, 2352, 2357, 2358, 2379, 2395, 2436, 2443, 2451, 2477, 2481, 2496, 2529, 2530, 2556, 2563, 2580, 2616, 2700, 2701, 2711, 2742, 2780, 2781, 2842, 2888, 2978, 3095, 3097, 3191, 3252, 3276, 3290, 3303, 3342, 3346, 3391, 3404, 3416, 3417, 3418, 3507, 3514, 3543, 3544, 3567, 3578, 3582, 3641, 3654, 3668, 3669, 3706, 3713, 3715, 3721, 3752, 3758, 3773, 3783, 3801, 3825, 3881, 3911, 3914, 3932, 3943, 3956, 4112, 4156, 4158, 4205, 4211, 4215, 4286, 4303, 4337, 4366, 4368, 4376, 4413, 4414, 4431, 4439, 4471, 4494, 4552, 4626, 4662, 4742, 4907, 5098, 5099, 5102, 5301, 5302, 5303, 5404, 5411, 5442, 5529, 5531],
                 'не белый фон': [3391, 3886, 5521, 5587, 5597, 5627, 5657, 5659, 5662, 5671, 5672, 5687, 5688, 5690, 5695, 5699, 5707, 5709, 5712, 5716, 5732, 5735, 5737, 5744, 5746, 5747, 5748, 5749, 5750, 5752, 5753, 5754, 5757, 5762, 5765, 5767, 5769, 5774, 5775, 5777, 5783, 5784, 5789, 5793, 5794, 5795, 5796, 5801, 5802, 5804, 5807, 5811, 5814, 5815, 5822, 5823, 5830, 5846, 5888, 5945, 5947, 5954, 5957, 5958, 5959, 5962, 5963, 5964, 5965, 5966, 5972, 5973, 5974, 5975, 5977, 5978, 5979],
                 'грязь': [4, 59, 61, 65, 67, 78, 79, 83, 93, 98, 105, 110, 120, 152, 171, 282, 329, 350, 413, 695, 1477, 1546, 1608, 1691, 1795, 1805, 1844, 1869, 1894, 1907, 2085, 2122, 2105, 2156, 2161, 2183, 2162, 2213, 2322, 2341, 2405, 2406, 2412, 2429, 2443, 2498, 2514, 2541, 2586, 2595, 2600, 2618, 2623, 2638, 2640, 2645, 2651, 2656, 2657, 2695, 2715, 2718, 2731, 2733, 2737, 2745, 2781, 2782, 2800, 2823, 2906, 3074, 3103, 3115, 3173, 3286, 3318, 3319, 3320, 3330, 3331, 3336, 3338, 3350, 3356, 3431, 3451, 3548, 3602, 3617, 3698, 3730, 3775, 3803, 3879, 3888, 3886, 3906, 3913, 3915, 3937, 3943, 3954, 3947, 4006, 4011, 4041, 4029, 4028, 4060, 4057, 4055, 4064, 4068, 4085, 4093, 4096, 4104, 4118, 4115, 4121, 4130, 4139, 4134, 4131, 4145, 4166, 4180, 4188, 4189, 4192, 4200, 4195, 4219, 4220, 4222, 4233, 4247, 4268, 4258, 4285, 4367, 4398, 4402, 4427, 4442, 4449, 4476, 4475, 4461, 4477, 4508, 4490, 4530, 4534, 4547, 4608, 4595, 4590, 4650, 4663, 4730, 4747, 4748, 4749, 4776, 4782, 4795, 4807, 4839, 4912, 4910, 4937, 4989, 4967, 5048, 5031, 5099, 5103, 5173, 5171, 5205, 5207, 5229, 5255, 5264, 5277, 5319, 5307, 5340, 5352, 5368, 5367, 5378, 5384, 5398, 5402, 5403, 5419, 5432, 5441, 5442, 5449, 5453, 5459, 5460, 5506, 5496, 5518, 5519, 5521, 5532, 5541, 5543, 5544, 5548, 5546, 5555, 5556, 5560, 5569, 5587, 5584, 5579, 5597, 5608, 5612, 5633, 5623, 5635, 5645, 5657, 5656, 5648, 5659, 5661, 5662, 5668, 5671, 5672, 5675, 5677, 5690, 5688, 5686, 5681, 5696, 5699, 5702, 5712, 5709, 5707, 5730, 5734, 5732, 5739, 5737, 5736, 5735, 5755, 5768, 5777, 5799, 5807, 5810, 5818, 5823, 5827, 5831, 5840, 5839, 5855, 5878, 5970, 5965, 5974],
                 'нестандартное форматирование текста': [87, 94, 148, 164, 178, 200, 201, 205, 214, 231, 237, 245, 251, 252, 259, 272, 330, 328, 310, 309, 390, 392, 603, 461, 456, 621, 679, 716, 743, 853, 953, 969, 1016, 1053, 1184, 1205, 1223, 1232, 1268, 1344, 1334, 1423, 1514, 1531, 1519, 1587, 1711, 1697, 1674, 1745, 1897, 1987, 1986, 2191, 2230, 2231, 2258, 2340, 2412, 2405, 2429, 2542, 2683, 2583, 2750, 2910, 2916, 3012, 3026, 3083, 3066, 3145, 3156, 3165, 3232, 3197, 3182, 3292, 3521, 3571, 3568, 3597, 3793, 3922, 3876, 3929, 3935, 4030, 4022, 4021, 4015, 4320, 4555, 4730, 4649, 5227, 5466, 5703, 5813, 5821],
                 'нестандартный контраст': [86, 94, 99, 115, 134, 148, 153, 155, 162, 164, 167, 173, 178, 186, 188, 201, 205, 237, 245, 252, 259, 267, 271, 272, 276, 277, 297, 298, 316, 326, 331, 344, 377, 392, 395, 404, 424, 426, 427, 446, 459, 468, 469, 480, 487, 498, 503, 537, 565, 569, 576, 579, 584, 585, 586, 587, 591, 600, 656, 666, 689, 703, 709, 741, 751, 753, 782, 785, 789, 805, 811, 812, 840, 852, 858, 877, 887, 894, 919, 949],
                 'без ошибок': [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 27, 28, 29, 30, 31, 36, 38, 43, 44, 45, 46, 47, 48, 51, 53, 54, 55, 57, 62, 63, 64, 66, 69, 72, 73, 76, 81, 84, 85, 89, 92, 95, 96, 97, 106, 111, 116, 117, 119, 124, 125, 126, 127, 128, 129, 130, 131, 132, 135, 136, 137, 139, 140, 141, 143, 144, 145, 146, 147, 149, 154, 156, 157, 158, 160, 161, 163, 165, 166, 168, 169, 170, 172, 174, 177, 179, 183, 185,]}

In [ ]:
for i, j in filenums_dict.items():
    print('{:3} - {}'.format(len(j), i))

In [ ]:
def get_img_path(base_path, filename):
    x = int(filename[:-4])
    lower_bound = (x // 100) * 100
    upper_bound = lower_bound + 100
    return base_path + f'{lower_bound}-{upper_bound}/' + filename


# Путь к папке с файлами на google диске
base_path = '/content/drive/MyDrive/ClearedReshapedBase/'

filenums_dict_2 = {i: [get_img_path(base_path, '{:04}.png'.format(
    k)) for k in j] for i, j in filenums_dict.items()}
filenums_dict_2